In [ ]:
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))

In [ ]:
import torch

In [ ]:
a=torch.tensor([[1,2,3],[2,3,4]])

In [ ]:
a.view(-1)

In [ ]:
rating_path=Path('..')/'ml-1m'/'ratings.dat'
rating_data, n_user, n_item=read_rating_data(rating_path)
cml=CML(n_user=n_user,n_item=n_item)
cml.init_setting(rating_data)
cml.data_setting()
cml.run()

In [ ]:
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix,dok_matrix,lil_matrix
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))
from pathlib import Path
import algo_common_func as ac
import numpy as np
from tqdm import tqdm
import torch
from torch.autograd import Variable
from timeit import default_timer as timer 

class CML:
    def __init__(self,n_user,n_item,n_factor=20,epoch=500,batch_size=500,div_size=0.2,margin=1):
        self.n_user=n_user
        self.n_item=n_item
        self.n_factor=n_factor
        self.epoch=epoch
        self.margin=margin
        self.train_set=None
        self.test_set=None
        self.div_size=div_size
        self.user_id=None
        self.item_id=None
        self.neg_item_id=None
        self.loss=None
        self.pred_distance=None
        self.pred_distance_neg=None
        self.clip_p=None
        self.clip_q=None
        self.train_data=None
        self.teset_data=None
        self.batch_size=batch_size
        self.csr_train=None
        self.csr_test=None
        self.test_user=None
        self.neg_items=None
        self.n_training=0
        self.n_batch=0
        self.test_dict=None
        self.P=None
        self.Q=None
        self.loss=None
        self.relu=None
        self.pair_dist=None
        
        
    def init_setting(self,rating_data):
        
        self.train_data, self.test_data=self.split_train_test(rating_data)
        self.csr_train, self.csr_test=self.make_csr(self.train_data,self.test_data)
        
        self.test_dict={}
        for user in range(self.n_user):
            self.test_dict[user]=self.csr_test.getrow(user).nonzero()[1]
        self.relu=torch.nn.ReLU()    
        
        self.pair_dist=torch.nn.PairwiseDistance(p=2)
        
        self.P=torch.nn.Parameter(torch.normal(mean=0,std=1/(self.n_factor**0.5),size=(self.n_user,self.n_factor)))
        self.Q=torch.nn.Parameter(torch.normal(mean=0,std=1/(self.n_factor**0.5),size=(self.n_item,self.n_factor)))
#         self.P=Variable(torch.normal(mean=0,std=1/(self.n_factor**0.5),size=(self.n_user,self.n_factor)),requires_grad=True)
#         self.Q=Variable(torch.normal(mean=0,std=1/(self.n_factor**0.5),size=(self.n_item,self.n_factor)),requires_grad=True)
    
        self.loss=torch.nn.MSELoss(reduction='sum') 

#         self.clip_p=tf1.assign(P,tf.clip_by_norm(P,1,axes=[1]))
#         self.clip_q = tf1.assign(Q, tf.clip_by_norm(Q, 1, axes=[1]))    
        
    def run(self):
        for epoch in range(self.epoch):
            # randomly choose user, movie id
            loss=self.train()
            start=timer()
            if epoch%5==0:
                
                hit_50,hit_100=self.test()
                if epoch%10==0:
                    print("loss ",loss)
                    print(str(epoch)+". "+str(timer()-start))
                    start=timer()
                    print(str(epoch+1)+". recall@50 : "+str(hit_50)+" , recall@100 : "+str(hit_100))
          
        
    def predict(self,u_neg_item,u_id):
        predict_val=self.pair_dist(self.P[u_id,:],self.Q[u_neg_item,:])
        return predict_val
    
    
    def recall_at_k(self,k,pred_item,test_item_id):
        k=k if len(test_item_id)>k else len(test_item_id)
        pred_item_lim=pred_item[:k]
        hit_id=np.intersect1d(pred_item_lim,test_item_id)
        return len(hit_id)/float(k)
    
    
    def test(self):
        ranked_arr={}
        hit_50=[]
        hit_100=[]
        for u in self.test_user:
            u_id=[]
            # movie id not rated by user,u
            u_neg_item=np.array(self.neg_items[u])
            u_id=np.repeat(u,len(u_neg_item))
            pre_rating=self.predict(u_neg_item,u_id)
            neg_item_index=np.array([u_neg_item,pre_rating.detach().numpy()]).T
            
            # 0 col : item id sorted by pred_rating
            # 1 col : predicted rating sorted
            ranked_arr[u]=neg_item_index[neg_item_index[:,1].argsort()[::-1]][:,0]
            # item id sorted
            pred_ratings_50=ranked_arr[u][:50]
            pred_ratings_100=ranked_arr[u][:100]
            hit_rate_50=self.recall_at_k(50,pred_ratings_50,self.test_dict[u])
            hit_rate_100=self.recall_at_k(100,pred_ratings_100,self.test_dict[u])
            
            if hit_rate_50 !=0:
                hit_50.append(hit_rate_50)
            if hit_rate_100 !=0:
                hit_100.append(hit_rate_100)
            
        return np.mean(hit_50), np.mean(hit_100)
    
    def train(self):
        train=self.csr_train.tocoo()
        row=train.row.reshape(-1)
        col=train.col.reshape(-1)
        # randomize the order
        index=np.random.permutation(self.n_training)
        #to list
        u_random=torch.tensor(list(row[index])).cuda()
        i_random=torch.tensor(list(col[index])).cuda()
        item_random_neg=[]
        
        # randomly choose one item id not rated by each user and put into list
        for user in u_random:
            n_random=np.random.randint(len(self.neg_items[user.item()]))
            item_random_neg.append(self.neg_items[user.item()][n_random])
            
        item_random_neg=torch.tensor(item_random_neg).cuda()    
            
        """
        train : loop for each batch
        """
        loss=0
    
        for b in range(self.n_batch):
            batch_user=u_random[b*self.batch_size:(b+1)*self.batch_size]
            batch_item=i_random[b*self.batch_size:(b+1)*self.batch_size]
            batch_item_neg=item_random_neg[b*self.batch_size:(b+1)*self.batch_size]
            optim=torch.optim.Adagrad([self.P,self.Q],lr=0.1)
            optim.zero_grad()
            loss=self.loss_m(batch_user,batch_item,batch_item_neg)
            loss.backward()
            optim.step()
            
        return loss
    
    
    def loss_m(self,user_id,item_id,neg_item_id):
        dist=self.relu(self.margin-self.pair_dist(self.P[user_id.tolist()],self.Q[neg_item_id.tolist()])+self.pair_dist(self.P[user_id.tolist()],self.Q[item_id.tolist()]))
        total_loss=torch.sum(dist)
        return total_loss
    
    
    def split_train_test(self,rating_data):
        """
        Split to train and test set 
        """
        train_data,test_data=train_test_split(rating_data,test_size=self.div_size)
        train_data[:,0]-=1
        train_data[:,1]-=1
        train_data[:,2]=1
        test_data[:,0]-=1
        test_data[:,1]-=1
        test_data[:,2]=1
        return train_data,test_data
    
    
    def make_csr(self,train_data,test_data):
        """
        return csr_matrx : train , test
        """
        return csr_matrix((train_data[:,2],(train_data[:,0],train_data[:,1]))), csr_matrix((test_data[:,2],(test_data[:,0],test_data[:,1])))
        
        
    def data_setting(self):
        
        self.n_training=len(self.train_data[:,0])
        self.n_batch=int(self.n_training/self.batch_size)
        
        #dictionary
        self.neg_items=self.neg_item_dic()
        self.test_user=set(self.test_data[:,0])
        
    def neg_item_dic(self):
        all_item=set(np.arange(self.n_item))
        neg_item={}
        for user_id in range(self.n_user):
            # 각 user마다 rating하지 않은 item을 list형태로 저장
            neg_item[user_id]=list(all_item-set(self.csr_train.getrow(user_id).nonzero()[1]))
        return neg_item
        